### Create Dataset From CSV and Folders

In [ ]:
import os
import glob
from dotenv import load_dotenv
import torch
import math
import shutil

from tqdm import tqdm
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import CSVLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders import DirectoryLoader


def main():
    """
    Main function to load Verilog and CSV data, create local embeddings, 
    and save a specific FAISS index.
    """
    # --- 1. Load Environment Variables ---
    load_dotenv()

    # --- 2. Define Paths ---
    DATASET_PATH = "../data/verilog_datasets"
    INDEX_PATH = os.path.join(DATASET_PATH, "faiss_qft_verieval")
    QFT_FOLDER_PATH = os.path.join(DATASET_PATH, "qft_pipelined")
    VERILOGEVAL_CSV_PATH = os.path.join(DATASET_PATH, "verilogeval-v2.csv")


    # --- 3. Clean up old database directory ---
    if os.path.exists(INDEX_PATH):
        print(f"--- Deleting old FAISS index directory: '{INDEX_PATH}' ---")
        try:
            shutil.rmtree(INDEX_PATH)
        except OSError as e:
            print(f"Error deleting directory {INDEX_PATH}: {e.strerror}")
            print("Please ensure no other programs are using this directory.")
            return
    print("--- Starting with a fresh index directory. ---")

    # --- 4. Load Documents ---
    all_docs = []
    
    # Load Verilog and Verilog Header files from the qft_pipelined directory
    print(f"\n--- Loading Verilog documents from '{QFT_FOLDER_PATH}' ---")
    if not os.path.exists(QFT_FOLDER_PATH):
        print(f"ERROR: QFT directory not found at '{QFT_FOLDER_PATH}'")
    else:
        # Load Verilog files as plain text, as the LanguageParser does not support Verilog
        loader_verilog = DirectoryLoader(
            QFT_FOLDER_PATH,
            glob=["**/*.v", "**/*.vh"], # UPDATED: Include .vh files
            loader_cls=TextLoader
        )
        verilog_docs = loader_verilog.load()
        all_docs.extend(verilog_docs)
        print(f"Successfully loaded {len(verilog_docs)} Verilog (.v & .vh) files.")

    # Load the specific CSV file
    print(f"\n--- Loading documents from '{VERILOGEVAL_CSV_PATH}' ---")
    if not os.path.exists(VERILOGEVAL_CSV_PATH):
        print(f"ERROR: CSV file not found at '{VERILOGEVAL_CSV_PATH}'")
    else:
        loader_csv = CSVLoader(
            file_path=VERILOGEVAL_CSV_PATH,
            source_column="instruction",
            csv_args={'delimiter': ',', 'quotechar': '"'}
        )
        try:
            csv_docs = loader_csv.load()
            all_docs.extend(csv_docs)
            print(f"Successfully loaded content from {len(csv_docs)} rows in the CSV.")
        except Exception as e:
            print(f"    - ERROR loading file {VERILOGEVAL_CSV_PATH}: {e}")

    if not all_docs:
        print("\nNo documents were loaded. Exiting.")
        return
    print(f"\nTotal documents loaded: {len(all_docs)}.")

    # --- 5. Split Documents into Chunks ---
    print("\n--- Splitting documents into smaller chunks ---")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    chunked_docs = text_splitter.split_documents(all_docs)
    print(f"Split the documents into {len(chunked_docs)} chunks.")

    # --- 6. Create Embeddings and Persist to FAISS ---
    print("\n--- Creating local embeddings and building FAISS index ---")
    
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name='all-MiniLM-L6-v2',
            model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
        )
        print(f"Embedding model loaded onto {'GPU' if torch.cuda.is_available() else 'CPU'}.")

        # --- ROBUST, TWO-STAGE INITIALIZATION FOR FAISS ---
        batch_size = 1024
        if not chunked_docs:
            print("No chunks to process. Exiting.")
            return

        # Stage 1: Initialize the vector store with the first batch.
        print("Initializing FAISS index with the first batch...")
        first_batch = chunked_docs[:batch_size]
        vectorstore = FAISS.from_documents(documents=first_batch, embedding=embeddings)

        # Stage 2: Add the rest in batches with a progress bar.
        remaining_chunks = chunked_docs[batch_size:]
        num_batches = math.ceil(len(remaining_chunks) / batch_size)
        for i in tqdm(range(num_batches), desc="Embedding and Storing in FAISS"):
            start = i * batch_size
            end = start + batch_size
            batch = remaining_chunks[start:end]
            if batch:
                vectorstore.add_documents(batch)
            
        print("\nSuccessfully created and populated FAISS index.")

    except Exception as e:
        print(f"\nAN UNEXPECTED ERROR OCCURRED: {e}")
        return

    # --- 7. Save Final Index ---
    print("\n--- Saving final FAISS index to disk ---")
    vectorstore.save_local(INDEX_PATH)
    print(f"FAISS index is stored in the folder: '{os.path.abspath(INDEX_PATH)}'")

if __name__ == "__main__":
    main()


### Create Dataset From Website

In [ ]:
import asyncio
import nest_asyncio
import os
import shutil
import math
from tqdm import tqdm
import torch
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

from crawl4ai import AsyncWebCrawler
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

async def main():
    """
    Main function to crawl the entire Lcapy documentation and create a FAISS index.
    """
    # --- 1. Define Configuration ---
    START_URL = "https://lcapy.readthedocs.io/en/latest/"
    TOC_LINK_SELECTOR = ".toctree-wrapper a"
    INDEX_PATH = "faiss_index_lcapy_docs"

    # --- 2. Clean up old index directory ---
    if os.path.exists(INDEX_PATH):
        print(f"--- Deleting old FAISS index directory: '{INDEX_PATH}' ---")
        shutil.rmtree(INDEX_PATH)
    print("--- Starting with a fresh index directory. ---")

    # --- 3. Discover all URLs using Requests and BeautifulSoup ---
    print(f"\n--- Step 1: Discovering all documentation URLs from '{START_URL}' ---")
    urls_to_crawl = [START_URL] # Always include the main page
    try:
        response = requests.get(START_URL)
        response.raise_for_status() # Will raise an error for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all link elements (<a>) that match the CSS selector
        links = soup.select(TOC_LINK_SELECTOR)
        
        for link in links:
            href = link.get('href')
            if href:
                # Convert relative links (e.g., 'netlists.html') to full URLs
                full_url = urljoin(START_URL, href)
                if full_url not in urls_to_crawl:
                    urls_to_crawl.append(full_url)
        
        print(f"Found {len(urls_to_crawl)} total pages to crawl.")

    except Exception as e:
        print(f"\nAN ERROR OCCURRED DURING URL DISCOVERY: {e}")
        return

    # --- 4. Crawl each discovered URL using crawl4ai ---
    print("\n--- Step 2: Crawling each page individually ---")
    all_docs = []
    try:
        async with AsyncWebCrawler() as crawler:
            # Use tqdm to show a progress bar for the crawling step
            for url in tqdm(urls_to_crawl, desc="Crawling Pages"):
                result = await crawler.arun(url=url)
                if result and result.markdown:
                    doc = Document(page_content=result.markdown, metadata={"source": result.url})
                    all_docs.append(doc)
    
        if not all_docs:
            print("\nCrawling did not return any documents. Exiting.")
            return

        print(f"\nSuccessfully crawled {len(all_docs)} pages.")

    except Exception as e:
        print(f"\nAN ERROR OCCURRED DURING CRAWLING: {e}")
        return

    # --- 5. Split Documents into Chunks ---
    print("\n--- Splitting documents into smaller chunks ---")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    chunked_docs = text_splitter.split_documents(all_docs)
    print(f"Split the documents into {len(chunked_docs)} chunks.")

    # --- 6. Create Embeddings and Persist to FAISS ---
    print("\n--- Creating local embeddings and building FAISS index ---")
    
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name='all-MiniLM-L6-v2',
            model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
        )
        print(f"Embedding model loaded onto {'GPU' if torch.cuda.is_available() else 'CPU'}.")

        batch_size = 512
        if not chunked_docs:
            print("No chunks to process. Exiting.")
            return

        print("Initializing FAISS index with the first batch...")
        first_batch = chunked_docs[:batch_size]
        vectorstore = FAISS.from_documents(documents=first_batch, embedding=embeddings)

        remaining_chunks = chunked_docs[batch_size:]
        num_batches = math.ceil(len(remaining_chunks) / batch_size)
        for i in tqdm(range(num_batches), desc="Embedding and Storing in FAISS"):
            start = i * batch_size
            end = start + batch_size
            batch = remaining_chunks[start:end]
            if batch:
                vectorstore.add_documents(batch)
            
        print("\nSuccessfully created and populated FAISS index.")

    except Exception as e:
        print(f"\nAN UNEXPECTED ERROR OCCURRED DURING INDEXING: {e}")
        return

    # --- 7. Save Final Index ---
    print("\n--- Saving final FAISS index to disk ---")
    vectorstore.save_local(INDEX_PATH)
    print(f"\n--- RAG data saved successfully! ---")
    print(f"FAISS index for Lcapy docs is stored in the folder: '{os.path.abspath(INDEX_PATH)}'")


if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main())

In [ ]:
import asyncio
import nest_asyncio
import os
import shutil
import math
from tqdm import tqdm
import torch
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

from crawl4ai import AsyncWebCrawler
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

async def main():
    """
    Main function to crawl the entire PySpice documentation and create a FAISS index.
    """
    # --- 1. Define Configuration ---
    START_URL = "https://pyspice.fabrice-salvaire.fr/releases/v1.5/index.html"
    TOC_LINK_SELECTOR = ".toctree-wrapper a"
    INDEX_PATH = "faiss_index_pyspice_docs"

    # --- 2. Clean up old index directory ---
    if os.path.exists(INDEX_PATH):
        print(f"--- Deleting old FAISS index directory: '{INDEX_PATH}' ---")
        shutil.rmtree(INDEX_PATH)
    print("--- Starting with a fresh index directory. ---")

    # --- 3. Discover all URLs using Requests and BeautifulSoup ---
    print(f"\n--- Step 1: Discovering all documentation URLs from '{START_URL}' ---")
    urls_to_crawl = [START_URL] # Always include the main page
    try:
        response = requests.get(START_URL)
        response.raise_for_status() # Will raise an error for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all link elements (<a>) that match the CSS selector
        links = soup.select(TOC_LINK_SELECTOR)
        
        for link in links:
            href = link.get('href')
            if href:
                # Convert relative links (e.g., 'netlists.html') to full URLs
                full_url = urljoin(START_URL, href)
                if full_url not in urls_to_crawl:
                    urls_to_crawl.append(full_url)
        
        print(f"Found {len(urls_to_crawl)} total pages to crawl.")

    except Exception as e:
        print(f"\nAN ERROR OCCURRED DURING URL DISCOVERY: {e}")
        return

    # --- 4. Crawl each discovered URL using crawl4ai ---
    print("\n--- Step 2: Crawling each page individually ---")
    all_docs = []
    try:
        async with AsyncWebCrawler() as crawler:
            # Use tqdm to show a progress bar for the crawling step
            for url in tqdm(urls_to_crawl, desc="Crawling Pages"):
                result = await crawler.arun(url=url)
                if result and result.markdown:
                    doc = Document(page_content=result.markdown, metadata={"source": result.url})
                    all_docs.append(doc)
    
        if not all_docs:
            print("\nCrawling did not return any documents. Exiting.")
            return

        print(f"\nSuccessfully crawled {len(all_docs)} pages.")

    except Exception as e:
        print(f"\nAN ERROR OCCURRED DURING CRAWLING: {e}")
        return

    # --- 5. Split Documents into Chunks ---
    print("\n--- Splitting documents into smaller chunks ---")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    chunked_docs = text_splitter.split_documents(all_docs)
    print(f"Split the documents into {len(chunked_docs)} chunks.")

    # --- 6. Create Embeddings and Persist to FAISS ---
    print("\n--- Creating local embeddings and building FAISS index ---")
    
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name='all-MiniLM-L6-v2',
            model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
        )
        print(f"Embedding model loaded onto {'GPU' if torch.cuda.is_available() else 'CPU'}.")

        batch_size = 512
        if not chunked_docs:
            print("No chunks to process. Exiting.")
            return

        print("Initializing FAISS index with the first batch...")
        first_batch = chunked_docs[:batch_size]
        vectorstore = FAISS.from_documents(documents=first_batch, embedding=embeddings)

        remaining_chunks = chunked_docs[batch_size:]
        num_batches = math.ceil(len(remaining_chunks) / batch_size)
        for i in tqdm(range(num_batches), desc="Embedding and Storing in FAISS"):
            start = i * batch_size
            end = start + batch_size
            batch = remaining_chunks[start:end]
            if batch:
                vectorstore.add_documents(batch)
            
        print("\nSuccessfully created and populated FAISS index.")

    except Exception as e:
        print(f"\nAN UNEXPECTED ERROR OCCURRED DURING INDEXING: {e}")
        return

    # --- 7. Save Final Index ---
    print("\n--- Saving final FAISS index to disk ---")
    vectorstore.save_local(INDEX_PATH)
    print(f"\n--- RAG data saved successfully! ---")
    print(f"FAISS index for PySpice docs is stored in the folder: '{os.path.abspath(INDEX_PATH)}'")


if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main())

### Crawl From Github

In [ ]:
import asyncio
from googlesearch import search
import nest_asyncio
from github import Github, UnknownObjectException
import re
import os
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# --- Helper Functions ---

def get_repo_name_from_url(url):
    """Extracts 'username/repository' from a GitHub URL."""
    match = re.search(r"github\.com/([^/]+/[^/]+)", url)
    if match:
        repo_name = match.group(1)
        # Remove .git suffix if present
        if repo_name.endswith('.git'):
            return repo_name[:-4]
        return repo_name
    return None

def get_repo_files_recursive(repo, path=""):
    """
    Recursively fetches the content of all text-based files in a repository.
    Returns a list of LangChain Document objects.
    """
    all_docs = []
    
    # Common text/code file extensions to include
    text_extensions = ['.v', '.sv', '.vhd', '.py', '.md', '.txt', '.c', '.h', '.cpp', '.hpp', '.js', '.html', '.css', '.json', '.xml']

    try:
        contents = repo.get_contents(path)
        for content_file in contents:
            if content_file.type == "dir":
                # If it's a directory, recurse into it
                print(f"  Entering directory: {content_file.path}")
                all_docs.extend(get_repo_files_recursive(repo, content_file.path))
            else:
                # If it's a file, check its extension
                if any(content_file.name.endswith(ext) for ext in text_extensions):
                    print(f"  Fetching file: {content_file.path}")
                    try:
                        # Decode content and create a LangChain Document
                        file_content = content_file.decoded_content.decode('utf-8')
                        doc = Document(page_content=file_content, metadata={"source": content_file.path})
                        all_docs.append(doc)
                    except Exception as e:
                        print(f"    - Could not decode file {content_file.path}: {e}")
    except Exception as e:
        print(f"Could not get contents for path '{path}'. It might be a submodule. Error: {e}")

    return all_docs


# --- Main Application Logic ---

async def main():
    query = "risc v verilog github"
    print(f"Searching for: {query}\n")

    # --- Step 1: Search and let user choose a repository ---
    try:
        urls = list(search(query, num_results=5, lang="en"))
        github_urls = [url for url in urls if "github.com" in url]
        if not github_urls:
            print("No GitHub repositories found in the top search results.")
            return

        print("Found the following GitHub repositories:")
        for i, url in enumerate(github_urls):
            print(f"  {i+1}: {url}")

        choice = -1
        while choice < 1 or choice > len(github_urls):
            try:
                user_input = input(f"\nPlease enter the number of the repo to process (1-{len(github_urls)}): ")
                choice = int(user_input)
            except ValueError:
                print("Invalid input. Please enter a number.")
        
        repo_name = get_repo_name_from_url(github_urls[choice - 1])
        if not repo_name:
            print("Could not extract a valid repository name.")
            return

        # --- Step 2: Recursively fetch all file contents from the repo ---
        print(f"\nInspecting repository: {repo_name}")
        g = Github() 
        repo = g.get_repo(repo_name)
        
        print("\n--- Starting to retrieve all files from repository ---")
        documents = get_repo_files_recursive(repo)
        
        if not documents:
            print("\nNo text-based files found to process.")
            return
            
        print(f"\nSuccessfully retrieved {len(documents)} files.")

        # --- Step 3: Chunk the documents ---
        print("\n--- Splitting documents into smaller chunks ---")
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        chunked_docs = text_splitter.split_documents(documents)
        print(f"Split the documents into {len(chunked_docs)} chunks.")

        # --- Step 4: Create embeddings and FAISS index ---
        print("\n--- Creating embeddings and building FAISS index (this may take a while) ---")
        # Use a popular, lightweight sentence-transformer model
        embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
        
        # This one command creates embeddings and the FAISS index
        vectorstore = FAISS.from_documents(chunked_docs, embeddings)
        print("Successfully created FAISS index from documents.")

        # --- Step 5: Save the FAISS index to disk ---
        save_path = f"faiss_index_{repo_name.replace('/', '_')}"
        vectorstore.save_local(save_path)
        print(f"\n--- RAG data saved successfully! ---")
        print(f"FAISS index and documents are stored in the folder: '{os.path.abspath(save_path)}'")
        print("You can now load this index for your RAG application.")

    except Exception as e:
        print(f"\nAn error occurred: {e}")

# Run the main asynchronous function
if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main())